### The Mandelbrot Set Iteration

The Mandelbrot set is given as the iteration of the following equations
$$
z_{n+1} = z^2_n + c
$$

Where:
* $z_0$ = 0 (starting value)
* $c$ is a complex number, where each unique $c$ will yield a different sequence of $z$ values

A point $c$ is in the Mandelbort set if, after iterating the equation multiple times, $|z|$ (the magnitude of $Z$) stays bounded (specifically, it remains $\leq$ 2). If $|z|$ escapes beyond 2, $c$ is not part of the set

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from multiprocessing import Pool
from worker import worker_function
from worker import worker_pure
import os
from multiprocessing import Pool
import sys
from worker import worker_function_sampling
from worker import worker_orthogonal
import pandas as pd

In [2]:

def mandelbrot(c_points, max_iter, escape_radius) -> tuple[np.array, np.array]:
    '''
    This function calculates the number of iterations until the magnitude of z escapes to infinity. 
    Within each iteration the z is updated with c, an imaginary number representing a grid point. 
    Ultimately, a mandelbrot is calculated. 
    '''
    iteration_count = np.zeros(c_points.shape)
    mandelbrot_set = np.zeros(c_points.shape, dtype=bool)
    for i in range(c_points.shape[0]):
        for j in range(c_points.shape[1]):

            # take a gridpoint
            c = c_points[i, j]
            z = 0
            for iteration in range(max_iter):

                # update of z
                z = z**2 + c
                if abs(z) > escape_radius:
                    mandelbrot_set[i, j] = False
                    iteration_count[i, j] =iteration
                    break
            else:
                mandelbrot_set[i, j] = True
                iteration_count[i, j] = max_iter
    return (mandelbrot_set, iteration_count)


            

# Plotting

#### Plotting different layouts of sampling methods

In [ ]:
from worker import get_example_samples


samples = 9 #root needs to be an int
random_seed = 57
(orth_samples, latin_samles, random_samples) = get_example_samples(samples, random_seed)

headers = ['Orthogonal', 'Latin Hypercube', 'Random', 'Deterministic' ]
fig, axes = plt.subplots(2, 2, figsize=(5, 5), sharey=True, sharex=True)

real = np.linspace(-2.0, 1.0, int(np.sqrt(samples)))
imag = np.linspace(-1.5, 1.5, int(np.sqrt(samples)))

real_grid, imag_grid = np.meshgrid(real, imag)
coordinates_det = np.array(list(zip(real_grid.flatten(), imag_grid.flatten())))


j = 0
i = 0
for k, coordinates in enumerate([orth_samples, latin_samles, random_samples, coordinates_det]):
    if j == 0:
        if i > 1: 
            j += 1
            i -= 2
    
    ax = axes[i][j]
    ax.scatter(coordinates[:, 0], coordinates[:, 1])
    ax.set_xlim(-2, 1)
    ax.set_ylim(-1.5, 1.5)

    ax.set_aspect('equal', adjustable='box')  # Ensures 1:1 aspect ratio

    ax.grid(which='minor', color='gray', linestyle='-', linewidth=0.5)   # Thinner minor grid

    ax.set_xticks(np.linspace(-2, 1, samples+1), minor=True)  
    ax.set_yticks(np.linspace(-1.5, 1.5, samples+1), minor=True)

    ax.grid(which='major', color='black', linestyle='-', linewidth=1.5)  # Thicker major grid every 3 cells
    ax.set_xticks(np.linspace(-2, 1, int(samples/np.sqrt(samples)) + 1, endpoint=True))  
    ax.set_yticks(np.linspace(-1.5, 1.5, int(samples/np.sqrt(samples))+1, endpoint=True))

    ax.set_title(headers[k])
    # ax.miminorticks_on()
    i+=1

fig.suptitle('Different Monte Carlo Sampling Methods', fontsize=14)
plt.savefig("Sampling_layout.png", dpi=300)

plt.show()

### Reading .txt file and processing data of the sampling methods

#### Plotting the deterministic results

In [ ]:
# plotting the results from deterministic sampling, Area as a function of the grid-size
df = pd.read_csv("question2.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])
df["fraction_mand"] = (1 - df["points_inside"]/df["total_points"])*9

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(4.3, 11))
for max_iter, group in df.groupby("max_iterations"):
    ax1.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')
    ax2.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')
    ax3.plot(group["grid_size"], group["fraction_mand"], label=f"{max_iter}", marker= 'o')

# full plot
ax1.set_ylabel("Area of Mandelbrot")
ax1.set_title("Full Size")
ax1.legend(title="Iteration Bound")
ax1.set_xlim(0, 5040)
ax1.grid(True)

# for max_iter, group in df.groupby("max_iterations"):
    

# zoomed in plot on the y-axes
ax2.set_ylabel("Area of Mandelbrot")
ax2.set_title("Zoomed In (y-Axes)")
ax2.set_xlim(0, 5040)
ax2.grid(True)
ax2.set_ylim(0.165*9, 0.1735*9)

# zooming in on x-axes
ax3.set_ylabel("Area of Mandelbrot")
ax3.set_xlabel("Grid Size")
ax3.set_title("Zoomed In (x-Axes)")
ax3.set_xlim(0, 5040)
ax3.grid(True)
ax3.set_xlim(0, 500)

fig.suptitle("Area of Mandelbrot vs Grid-Size")
plt.tight_layout()
plt.savefig("Area_deterministic.png", dpi=300)
plt.show()


In [ ]:
import pandas as pd

df_o = pd.read_csv("orthogonal.txt", header=1)  

# Assign column labels
df_o.columns = ["grid_size", "max_iterations", "run", "total_points", "points_inside"]
# calculating area based on fraction of points inside and outside
df_o['fraction'] = (1- df_o['points_inside'] / df_o['total_points'])*9

# calculating variance and mean of area
average_fraction_o = df_o.groupby(['grid_size', 'max_iterations'])['fraction'].mean()
variance_fraction_o = df_o.groupby(['grid_size', 'max_iterations'])['fraction'].var()

# saving the variance and mean of different groups for orthogonal in a new dataframe 
mean_var_o = pd.DataFrame({
    'mean': average_fraction_o,
    'variance': variance_fraction_o
}).reset_index()


colors = ["orange", "green", "purple"]
fig, (ax1, ax2) = plt.subplots(2,1, sharex= True, figsize=(5,5), gridspec_kw={'height_ratios': [1, 1], 'hspace': 0.05})
# bax = brokenaxes(ylims=((1.5, 1.525), (1.58, 1.595)), hspace=.05) 
i = 0
for name, group in mean_var_o.groupby("max_iterations"):
    ax1.plot(group['grid_size'], group['mean'], color=colors[i], label=f"Iteration Bound: {name}")
    ax1.errorbar(group['grid_size'], group["mean"], yerr=np.sqrt(group['variance']), fmt='o-', color=colors[i], capsize=5)

    # Plot on ax2 without labels to avoid duplicating in the legend
    ax2.plot(group['grid_size'], group['mean'], color=colors[i])
    ax2.errorbar(group['grid_size'], group["mean"], yerr=np.sqrt(group['variance']), fmt='o-', color=colors[i], capsize=5)

    i += 1

ax2.plot(group['grid_size'], [1.506484]*4, color='black', linestyle= '--', label='True Value')
ax1.plot(group['grid_size'], [1.506484]*4, color='black', linestyle= '--', label='True Value')
ax1.legend()
# ax2.legend()

ax1.set_ylim(1.58, 1.605)
ax2.set_ylim(1.5, 1.525)

# Set log scale on x-axis
ax1.set_xscale("log")
ax2.set_xscale("log")

custom_ticks = [100, 500, 1000, 5000]
ax2.set_xticks(custom_ticks)  # Set tick positions
ax2.set_xticklabels([f"${tick}^2$" for tick in custom_ticks])
ax2.set_xlabel("Sample Size")
fig.suptitle("Orthogonal Sampling over 10 Runs")
plt.tight_layout()
plt.savefig("orthogonal_mean.png", dpi=300)
plt.show()

In [ ]:
import pandas as pd


df_l = pd.read_csv("latin.txt", header=1)  # Set header=None since there is no header in the file

# Assign column labels
df_l.columns = ["grid_size", "max_iterations", "run", "total_points", "points_inside"]
df_l['fraction'] = (1- df_l['points_inside'] / df_l['total_points'])*9

average_fraction = df_l.groupby(['grid_size', 'max_iterations'])['fraction'].mean()
variance_fraction = df_l.groupby(['grid_size', 'max_iterations'])['fraction'].var()

# creating a data frame with mean and variance of every group for latin hypercube
mean_var_l = pd.DataFrame({
    'mean': average_fraction,
    'variance': variance_fraction
}).reset_index()
print(mean_var_l)

colors = ["orange", "green", "purple"]
fig, (ax1, ax2) = plt.subplots(2,1, sharex= True, figsize=(5,5), gridspec_kw={'height_ratios': [1, 1], 'hspace': 0.05})
i = 0
for name, group in mean_var_l.groupby("max_iterations"):
    ax1.plot(group['grid_size'], group['mean'], color=colors[i], label=f"Iteration Bound: {name}")
    ax1.errorbar(group['grid_size'], group["mean"], yerr=np.sqrt(group['variance']), fmt='o-', color=colors[i], capsize=5)

    # Plot on ax2 without labels to avoid duplicating in the legend
    ax2.plot(group['grid_size'], group['mean'], color=colors[i])
    ax2.errorbar(group['grid_size'], group["mean"], yerr=np.sqrt(group['variance']), fmt='o-', color=colors[i], capsize=5)

    i += 1

# plotting true value
ax2.plot(group['grid_size'], [1.506484]*4, color='black', linestyle= '--', label='True Value')
ax1.plot(group['grid_size'], [1.506484]*4, color='black', linestyle= '--', label='True Value')
ax1.legend()

# Plot finetuning
ax1.set_ylim(1.56, 1.63)
ax2.set_ylim(1.48, 1.55)
ax1.set_xscale("log")
ax2.set_xscale("log")

custom_ticks = [100, 500, 1000, 5000]
ax2.set_xticks(custom_ticks)  # Set tick positions
ax2.set_xticklabels([f"${tick}^2$" for tick in custom_ticks])
ax2.set_xlabel("Sample Size")
fig.suptitle("Latin Hypercube Sampling over 10 Runs")
plt.tight_layout()
plt.savefig("Latin_mean.png", dpi=300)
plt.show()


In [ ]:

df_r = pd.read_csv("random.txt", header=1)  # Set header=None since there is no header in the file

# Assign column labels
df_r.columns = ["grid_size", "max_iterations", "run", "total_points", "points_inside"]

# calculate area with fraction inside and outside mandelbrot
df_r['fraction'] = (1- df_r['points_inside'] / df_r['total_points'])*9


# calculating fraction and variance
average_fraction = df_r.groupby(['grid_size', 'max_iterations'])['fraction'].mean()
variance_fraction = df_r.groupby(['grid_size', 'max_iterations'])['fraction'].var()

# creating data frame of mean and variance of data from different groups within random sampling
mean_var = pd.DataFrame({
    'mean': average_fraction,
    'variance': variance_fraction
}).reset_index()

colors = ["orange", "green", "purple"]
fig, ax = plt.subplots(1,1, sharex= True, figsize=(4.8,4.8))
i = 0

# plot mean and std for every line
for name, group in mean_var.groupby("max_iterations"):
    ax.plot(group['grid_size'], group['mean'], color=colors[i], label=f"Iteration Bound: {name}")
    ax.errorbar(group['grid_size'], group["mean"], yerr=np.sqrt(group['variance']), fmt='o-', color=colors[i], capsize=5)
    i += 1

ax.plot(group['grid_size'], [1.506484]*4, color='black', linestyle= '--', label='True Value')
ax.legend()
ax.set_xscale("log")

custom_ticks = [100, 500, 1000, 5000]
ax.set_xticks(custom_ticks)  # Set tick positions
ax.set_xticklabels([f"${tick}^2$" for tick in custom_ticks])
ax.set_xlabel("Sample Size")
fig.suptitle("Random Sampling over 10 Runs")
plt.tight_layout()
plt.savefig("random_sampling.png", dpi=300)
plt.show()



#### Plotting variances

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(4.5, 9), sharex=True)
fig.suptitle("Variance for Monte Carlo Sampling")

# Plot for Random Sampling

for j, m_v in enumerate([mean_var, mean_var_l, mean_var_o]):
    i = 0
    for name, group in m_v.groupby("max_iterations"):
        axes[j].plot(group["grid_size"], group['variance'], marker='o', color=colors[i], label=f'Iteration Bound {name}')
        i += 1
    # axes[j].set_xlabel("Sample Size")
    axes[j].set_yscale("log")
    axes[j].set_ylabel("Variance")

axes[0].set_title("Random Sampling")
axes[0].legend()
axes[1].set_title("Latin Hyper-Cube Sampling")
axes[2].set_title("Orthogonal Sampling")

axes[2].set_xlabel("Sample Size")
axes[2].set_xticks([100, 500, 1000, 5000])
axes[2].set_xticklabels([f"${tick}^2$" for tick in [100, 500, 1000, 5000]], rotation=45)
plt.tight_layout()  # Adjust layout to fit the main title
plt.savefig('variances.png', dpi=300)
plt.show()


#### Plotting mean absolute errors

In [ ]:
df = pd.read_csv("question2.txt", delimiter=",", skiprows=1,
                 names=["grid_size", "max_iterations", "total_points", "points_inside"])
df["fraction"] = (1 - df["points_inside"]/df["total_points"])*9
true_value = [1.506484]*len(df['fraction'])
df["diff_true"] = np.abs(true_value-df['fraction'])


true_value = [1.506484]*len(df_l['fraction'])
df_l["diff_true"] = np.abs(true_value-df_l['fraction'])
average_diff_l = df_l.groupby(['grid_size', 'max_iterations'])['diff_true'].mean()
variance_diff_l = df_l.groupby(['grid_size', 'max_iterations'])['diff_true'].var()


true_value = [1.506484]*len(df_o['fraction'])
df_o["diff_true"] = np.abs(true_value-df_o['fraction'])

average_diff_o = df_o.groupby(['grid_size', 'max_iterations'])['diff_true'].mean()
variance_diff_o = df_o.groupby(['grid_size', 'max_iterations'])['diff_true'].var()

true_value = [1.506484]*len(df_r['fraction'])
df_r["diff_true"] = np.abs(true_value-df_r['fraction'])

average_diff_r = df_r.groupby(['grid_size', 'max_iterations'])['diff_true'].mean()
variance_diff_r = df_r.groupby(['grid_size', 'max_iterations'])['diff_true'].var()

mean_var_diff = pd.DataFrame({
    'mean_o': average_diff_o,
    'variance_o': variance_diff_o,
    'mean_l': average_diff_l,
    'variance_l': variance_diff_l, 
    'mean_r': average_diff_r,
    'variance_r': variance_diff_r
}).reset_index()


grid_sizes = [100, 500, 1000, 5000]
grid_sub_df = df[(df['grid_size'].isin(grid_sizes))].reset_index()

fig, axes = plt.subplots(3, 1,sharex=True,figsize=(6,9.5), gridspec_kw={'height_ratios': [1.4, 1, 1]})
i = 0
for name, group in mean_var_diff.groupby("max_iterations"):
    ax = axes[i]
    ax.plot(group['grid_size'], group['mean_l'], marker='s', color=colors[i], linestyle='--', label=f"Latin-Hypercube Sampling")
    # ax.errorbar(group['grid_size'], group["mean_l"], yerr=np.sqrt(group['variance_l']), fmt='o', color=colors[i], capsize=5)

    ax.plot(group['grid_size'], group['mean_r'], marker='v', color=colors[i], linestyle=':', label=f"Random Sampling")
    # ax.errorbar(group['grid_size'], group["mean_r"], yerr=np.sqrt(group['variance_l']), fmt='o', color=colors[i], capsize=5)

    ax.plot(group['grid_size'], group['mean_o'], marker='d', color=colors[i], linestyle='-.', label=f"Orthogonal Sampling")
    # ax.errorbar(group['grid_size'], group["mean_o"], yerr=np.sqrt(group['variance_o']), fmt='o', color=colors[i], capsize=5)

    deterministic_val= grid_sub_df[grid_sub_df["max_iterations"] == name]
    ax.plot(group['grid_size'], deterministic_val['diff_true'], marker='o', color=colors[i], label='Deterministic Sampling')

    ax.set_xscale("log")
    ax.set_ylabel("Mean Error")
    ax.set_title(f"Iteration Bound: {name}", fontsize=10)
    ax.legend()
    i += 1

axes[2].set_xlim(95, 5100)
custom_ticks = [100, 500, 1000, 5000]
axes[2].set_xticks(custom_ticks)  # Set tick positions
axes[2].set_xticklabels([f"${tick}^2$" for tick in custom_ticks])
axes[2].set_xlabel("Number of Samples")
fig.suptitle("Abosolute Mean Error with True value")
plt.tight_layout()

plt.savefig('difference_act_value.png', dpi=300)
plt.show()

# Implementing Sampling methods

#### Framework for Parallelized implementation -> for all sampling
Do not forget to add sampling specific code in the worker.py file, and add parameters underneath.   
Code is currently running with deterministic function mandelbrot in worker.py. Idea is to replace this function (do not remove mandelbrot tho) with sampling specific function.  
Also, in case of changes in worker.py, reload this module, such that notebook refreshes imports and notices changes.  

Copy this cell, to keep this framework reusable


In [ ]:
import os
from multiprocessing import Pool
# Import own fucntion from .py file -> write for own sampling method! parameters of this function = grid (already given), max iteration bound (already given), sampling specific parameters (need to be difined still)
# Don't forget to also return the parameter values (as specified in example worker function)
from worker import worker_function


def partition_func(pars):

    # max 10 processes
    PROCESSES = 10 
    dict_res = {}
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        results = pool.map(worker_function,  pars)
        for res in results:
            (tot_points, out_points), par = res
            dict_res[par] = (tot_points, out_points)
            print(f"Gridsize: {par[0]}, Iteration Bound:{par[1]}\nTotal points and points inside mandelbrot: {tot_points, out_points}")
    return dict_res

if __name__ == '__main__':
    # Specify the s_values and grid sizes you want to experiment with
    # add sampling specific parameters, which are used in the worker method. (such as number of random samples taken (suggestion: choose 3 values for this, low, med, high))
    # evaluate when stochastic method is better than deterministic one. 
    
    # These are the values to experiment with, test with dummy values (these are quicker)
    s_values = [50, 500, 2000]
    grids = [100, 1000, 5000]


    # Dummy values
    s_values = [10, 50, 60, 70]
    grids = [10, 50, 100, 150]

    par_combos = []

    # every proc gets a full grid, every proc a different parameter set (defined above)
    # add an extra for loop for an extra parameter
    for g in grids:
        real = np.linspace(-2.0, 1.0, g)
        imag = np.linspace(-1.5, 1.5, g)
        real_grid, imag_grid = np.meshgrid(real, imag)
        c_points = real_grid + 1j * imag_grid
        for s in s_values:
            par_combos.append((c_points, g, s))
    saved_values = {}
    # make sure the number of procs you use does not exceed the processor count

    print(f"Number of available CPU cores: {os.cpu_count()}")
    for i in range(10):
        # provide random seed such that random function will be different
        random_seed = i+42
        par_combos = [(comb, random_seed) for comb in par_combos]
        saved_values = partition_func(par_combos)

        # ensure saving the values with this code:
        # CHANGE FILE NAME!!
        with open("question3-1.txt", "w") as file:
            # add parameter in case a parameter is added to the 
            if i == 0:
                file.write("grid_size, max_iterations, total_points, points_inside\n")
            file.write(f"run {i}:")
            for (key1, key2), (value1, value2)  in saved_values.items():
                file.write(f"{key1}, {key2}, {value1}, {value2}\n")


#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

#### Orthongonal Sampling

In [ ]:

def partition_func(pars):
    '''
    This function parallelizes the simulation, by assigning different proc different parametersets. 
    It utilizes asynchronic parallelization, meaning the output is printed after the task is finished
    '''
    PROCESSES = 10 
    dict_res = {}
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        print("Starting parallel execution")
        # asynchronyc parallelization
        for res in pool.imap_unordered(worker_orthogonal,  pars):
            (tot_points, out_points), par = res
            dict_res[par] = (tot_points, out_points)
            print(f"Gridsize: {par[0]}, Iteration Bound:{par[1]}, Run: {par[2]}\n Total points and points inside mandelbrot: {tot_points, out_points}", flush=True)
    return dict_res

if __name__ == '__main__':
    
    # These are the values to experiment with, test with dummy values (these are quicker)
    # s_values = [50, 500, 2000]
    # grids = [100, 500, 1000, 5000]

    # Dummy values
    s_values = [10, 50, 60, 70]
    grids = [7, 31, 71, 101]
    par_combos = []

    # make sure the number of procs you use does not exceed the processor count
    print(f"Number of available CPU cores: {os.cpu_count()}")
    
    # 10 runs
    for i in range(10):
        # provide random seed such that random function will be different
        random_seed = i+42

        # every proc gets a full grid, every proc a different parameter set (defined above)
        for g in grids:
            for s in s_values:
                par_combos.append( (g, s, i, random_seed))

    saved_values = {}
    saved_values = partition_func(par_combos)

    # ensure saving the values with this code:
    with open("orthogonal_dummy.txt", "w") as file:
        file.write("grid_size, max_iterations, run, total_points, points_inside\n")
        for (key1, key2, run), (value1, value2)  in saved_values.items():
            file.write(f"{key1}, {key2}, {run}, {value1}, {value2}\n")


In [ ]:


def partition_func(pars):
    '''
    This function parallelizes the simulation, by assigning different proc different parametersets. 
    It utilizes asynchronic parallelization, meaning the output is printed after the task is finished
    '''
    # max 10 processes
    PROCESSES = 10 
    dict_res = {}
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        print("Starting parallel execution")
        for res in pool.imap_unordered(worker_function_sampling,  pars):
            (tot_points, out_points), par = res
            dict_res[par] = (tot_points, out_points)
            print(f"Gridsize: {par[0]}, Iteration Bound:{par[1]}, Run: {par[2]}\n Total points and points inside mandelbrot: {tot_points, out_points}", flush=True)
    return dict_res

if __name__ == '__main__':
    
    # These are the values to experiment with, test with dummy values (these are quicker)
    s_values = [50, 500, 2000]
    grids = [100, 500, 1000, 5000]

    # Dummy values
    s_values = [10, 50, 60, 70]
    grids = [7, 31, 71, 101]

    par_combos = []
    
    print(f"Number of available CPU cores: {os.cpu_count()}")
    
    # 10 runs
    for i in range(10):
        # provide random seed such that random function will be different
        random_seed = i+42

        # every proc gets a full grid, every proc a different parameter set (defined above)
        # add an extra for loop for an extra parameter
        for g in grids:
            
            # print(c_points)
            for s in s_values:
                par_combos.append( (g, s, i, random_seed))

    saved_values = {}
    saved_values = partition_func(par_combos)

    # ensure saving the values with this code:
    with open("latin-dummy.txt", "w") as file:
        # add parameter in case a parameter is added to the 
        file.write("grid_size, max_iterations, run, total_points, points_inside\n")
        for (key1, key2, run), (value1, value2)  in saved_values.items():
            file.write(f"{key1}, {key2}, {run}, {value1}, {value2}\n")


#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

In [ ]:
# grid_size = np.linspace(4, )
import os
from multiprocessing import Pool
from worker import worker_function

def partition_grid(grid, proc):
    grid_size = grid//proc
    rest = grid % proc

    indexes =[]
    index = 0

    real = np.linspace(-2.0, 1.0, grid)
    imag = np.linspace(-1.5, 1.5, grid)

    # Create a 2D grid of complex numbers c
    real_grid, imag_grid = np.meshgrid(real, imag)
    c_points = real_grid + 1j * imag_grid

    for _ in range(proc):
        if rest > 0:
            addit = 1
            rest -=1
        else: 
            addit = 0
        indexes.append((index, index+grid_size+addit))
        index += grid_size + addit
    
    c_slices = [c_points[start:end] for start, end in indexes]
    return c_slices


def driver_func(grid, s):

    # max 10 processes
    PROCESSES = 10 
    with Pool(PROCESSES) as pool:  # Adjust the number of processes as needed 
        pid = os.getpid()
        # print(f"Current Process ID: {pid}")

        # every process gets a fraction of the grid (row-wise partitioning)
        c_parts = partition_grid(grid, PROCESSES)

        args = [(c_slice, s) for c_slice in c_parts]
        results = pool.map(worker_function,  args)
        total_points_parts, inside_points_parts = zip(*results)
        total_points = sum(total_points_parts)
        inside_points= sum(inside_points_parts)
        print(f"Result: {results}\nTotal points and points inside mandelbrot: {total_points, inside_points}")
    return total_points, inside_points

if __name__ == '__main__':
    s_values = [10, 20, 50, 100, 150, 250, 500, 1000, 2000]
    grids = [10, 50, 100, 250, 500, 1000, 2000, 5000]
    
    saved_values = {}
    # make sure the number of procs you use does not exceed the processor count
    print(f"Number of available CPU cores: {os.cpu_count()}")
    for grid in grids:
        for s_value in s_values:
            key = (grid, s_value)
            tot_points, in_points = driver_func(grid, s_value)
            saved_values[key] = (tot_points, in_points)
            
    with open("question2.txt", "w") as file:
        file.write("grid_size, max_iterations, total_points, points_inside\n")
        for (key1, key2), (value1, value2)  in saved_values.items():
            file.write(f"{key1}, {key2}, {value1}, {value2}\n")

#Since the mandelbrot is symmetrical, we could halve the grid and get the same fraction 

## Random Sampling 

In [ ]:
def partition_random(pars):
    '''
    This function parallelizes the simulation, by assigning different proc different parametersets. 
    It utilizes asynchronic parallelization, meaning the output is printed after the task is finished
    '''
    PROCESSES = 10
    dict_res = {}
    with Pool(PROCESSES) as pool:
        for res in pool.imap_unordered(worker_pure, pars):
            (tot_points, in_points), par = res
            dict_res[par] = (tot_points, in_points)
            estimated_area = (1-(in_points / tot_points)) * 9
            print(f"Grid-Size: {par[0]}, Iterations: {par[1]}, run: {par[2]}, Area: {estimated_area:.6f}")
    return dict_res

if __name__ == "__main__":
    # variables for experimentation
    s_values = [50, 500, 2000]
    grids = [100, 500, 1000, 5000]

    # dummy values
    s_values = [50, 100, 200]
    grids = [100, 200, 300]

    par_combos = []
    for i in range(10):
        for size in grids:
            for max_iter in s_values:
                
                # update rand for uniqueness across different runs
                rand = 42+i
                par_combos.append((size, max_iter, i, rand))
    print(f"Number of available CPU cores: {os.cpu_count()}")
    saved_values = partition_random(par_combos)

    #save values
    with open('random_dummy.txt', 'w') as file:
        file.write('grid_size, max_iterations, run, total_points, points_inside\n')
        for (key1, key2, key3), (value1, value2) in saved_values.items():
            file.write(f"{key1}, {key2}, {key3}, {value1}, {value2}\n")


#### Making a Mandelbrot figure

In [ ]:
# Setting up the imaginary point set
real = np.linspace(-2.0, 1.0, 1000)
imag = np.linspace(-1.5, 1.5, 1000)

# Create a 2D grid of complex numbers c
real_grid, imag_grid = np.meshgrid(real, imag)
c_points = real_grid + 1j * imag_grid

max_iter = 250
escape_radius = 2

mandel_set, iter_count = mandelbrot(c_points, max_iter, escape_radius)

plt.figure(figsize=(12, 8), dpi=300)
plt.imshow(iter_count, extent=(-2.0, 1.0, -1.5, 1.5), cmap='plasma', origin='lower')
plt.xlabel("Re(c)")
plt.ylabel("Im(c)")
plt.title("Mandelbrot Set")
plt.colorbar(label="In Mandelbrot Set")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import colorsys

def mandelbrot_iteration(c, max_iter, escape_radius):
    z = 0
    for n in range(max_iter):
        z = z*z + c
        if abs(z) > escape_radius:
            return n
    return max_iter

def compute_mandelbrot(real, imag, max_iter, escape_radius):
    height, width = len(imag), len(real)
    iteration_count = np.zeros((height, width), dtype=np.int64)
    
    # Create a mesh grid for vectorized computation
    real_grid, imag_grid = np.meshgrid(real, imag)
    c = real_grid + 1j * imag_grid
    z = np.zeros_like(c, dtype=np.complex128)
    
    for i in range(max_iter):
        mask = np.abs(z) <= escape_radius
        z[mask] = z[mask]**2 + c[mask]
        iteration_count[mask] = i
    
    # Set points that never escaped to max_iter
    iteration_count[np.abs(z) <= escape_radius] = max_iter
    
    return iteration_count

def create_custom_colormap():
    # Create colors for our custom colormap
    colors = []
    for i in range(256):
        # Convert HSV to RGB, cycling through hues
        hue = i/256
        saturation = 1.0
        value = 1.0 if i > 10 else i/10.0  # Darker colors for low iteration counts
        rgb = colorsys.hsv_to_rgb(hue, saturation, value)
        colors.append(rgb)
    
    # Add black for the Mandelbrot set itself
    colors.append((0, 0, 0))
    
    return LinearSegmentedColormap.from_list('custom', colors)

def plot_mandelbrot(iteration_count, real_range, imag_range, max_iter, 
                   zoom_center=None, zoom_width=None, title="Enhanced Mandelbrot Set"):
    plt.figure(figsize=(15, 10), dpi=300)
    
    # Create the plot with a custom colormap
    custom_cmap = create_custom_colormap()
    
    # Normalize iteration counts for smoother color transitions
    smooth_iter = iteration_count + 1 - np.log(np.log(np.abs(iteration_count)))/np.log(2)
    smooth_iter[iteration_count == max_iter] = max_iter
    
    # Plot with enhanced aesthetics
    plt.imshow(smooth_iter, 
              extent=(real_range[0], real_range[-1], imag_range[0], imag_range[-1]),
              cmap=custom_cmap,
              origin='lower',
              aspect='equal')
    
    # Add gridlines
    plt.grid(True, alpha=0.3, linestyle='--')
    
    # Enhance labels and title
    plt.xlabel("Re(c)", fontsize=12)
    plt.ylabel("Im(c)", fontsize=12)
    plt.title(title, fontsize=14, pad=20)
    
    # Add colorbar with custom label
    cbar = plt.colorbar(label="Iteration Count", pad=0.02)
    cbar.ax.set_ylabel("Iteration Count", fontsize=10)
    
    # Add zoom box if specified
    if zoom_center and zoom_width:
        zoom_rect = plt.Rectangle(
            (zoom_center[0] - zoom_width/2, zoom_center[1] - zoom_width/2),
            zoom_width, zoom_width,
            fill=False, color='white', linestyle='--'
        )
        plt.gca().add_patch(zoom_rect)
    
    plt.tight_layout()
    return plt.gcf()

# Parameters for the main plot
# Reduced resolution for faster computation since we don't have Numba
real = np.linspace(-2.0, 1.0, 1000)  
imag = np.linspace(-1.5, 1.5, 1000)
max_iter = 1000
escape_radius = 2.0

# Compute the main Mandelbrot set
iteration_count = compute_mandelbrot(real, imag, max_iter, escape_radius)

# Create main plot
main_plot = plot_mandelbrot(iteration_count, real, imag, max_iter, 
                           title="The Mandelbrot Set")
plt.show()

# Create a zoomed plot of an interesting region
zoom_center = (-0.7435, 0.1314)
zoom_width = 0.002

# Calculate new ranges for zoom
zoom_real = np.linspace(zoom_center[0] - zoom_width/2, 
                       zoom_center[0] + zoom_width/2, 1000)
zoom_imag = np.linspace(zoom_center[1] - zoom_width/2, 
                       zoom_center[1] + zoom_width/2, 1000)

# Compute zoomed region
zoom_iteration_count = compute_mandelbrot(zoom_real, zoom_imag, max_iter, escape_radius)

# Create zoomed plot
zoom_plot = plot_mandelbrot(zoom_iteration_count, zoom_real, zoom_imag, max_iter,
                           title=f"Mandelbrot Set Zoom (width={zoom_width:.6f})")
plt.show()